# Create PI vectorstore with Chroma DB

Import necessary libraries

In [2]:
import os
import json
import pandas as pd
import numpy as np
import re
import os

from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
EDIDP-CSAMN-EDICT-2020 proposal 
Cyber4de-searchaMicrosoft Azure Sentinel
PR-0021-21-MAISTRA_DS + PoC_Tabelau
Vulcan-Cyber-Mapping-CVEs-to-MITRE.pdf
Uputa za organizaciju službenih putovanja i obračun putnih naloga.pdf',
AI-Maturity-and-Organizations-eBook.pdf',
Data Science 101 Statistics Overview.pdf',
ODLU_04_2023_Odluka o dodjeli službenih mobitela 2023.pdf',
PRAV_111_o izmjenama i dopunama Pravilnika o radu.pdf',
PRAV_122_Pravilnik o korištenju budžeta za edukacije u 2023.pdf',
PRAV_129_Pravilnik o službenim putovanjima_FV.pdf',
PRAV_99_Pravilnik o radu 1017.pdf'
ATNVSP 2023 Lecture 3 - Analytical appliances.pptx
ATNVSP 2023 Lecture 2 - Data Lake concept.pptx
data/HT_report.pdf

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (3045497258.py, line 3)

# Document Loading
Load the PDF document containing the financial report

In [3]:
# list only pdf file paths in data folder
pdf_file_paths = [os.path.join('../data/New folder/', f) for f in os.listdir('../data/New folder/') if f.endswith('.pdf')]
pdf_file_paths

['../data/New folder/2017.06.14._Uputa za organizaciju službenih putovanja i obračun putnih naloga.pdf',
 '../data/New folder/AI-Maturity-and-Organizations-eBook.pdf',
 '../data/New folder/Data Science 101 Statistics Overview.pdf',
 '../data/New folder/ODLU_04_2023_Odluka o dodjeli službenih mobitela 2023.pdf',
 '../data/New folder/PRAV_111_o izmjenama i dopunama Pravilnika o radu.pdf',
 '../data/New folder/PRAV_122_Pravilnik o korištenju budžeta za edukacije u 2023.pdf',
 '../data/New folder/PRAV_129_Pravilnik o službenim putovanjima_FV.pdf',
 '../data/New folder/PRAV_99_Pravilnik o radu 1017.pdf']

In [5]:
print(pdf_file_paths)

['../data/2017.06.14._Uputa za organizaciju službenih putovanja i obračun putnih naloga.pdf', '../data/AI-Maturity-and-Organizations-eBook.pdf', '../data/Data Science 101 Statistics Overview.pdf', '../data/ODLU_04_2023_Odluka o dodjeli službenih mobitela 2023.pdf', '../data/PRAV_111_o izmjenama i dopunama Pravilnika o radu.pdf', '../data/PRAV_122_Pravilnik o korištenju budžeta za edukacije u 2023.pdf', '../data/PRAV_129_Pravilnik o službenim putovanjima_FV.pdf', '../data/PRAV_99_Pravilnik o radu 1017.pdf']


In [6]:
# list only pdf files in "../data/ foler
pdf_files = os.listdir("../data/")
pdf_files = [f for f in pdf_files if f.endswith(".pdf")]
print(pdf_files)


['2017.06.14._Uputa za organizaciju službenih putovanja i obračun putnih naloga.pdf', 'AI-Maturity-and-Organizations-eBook.pdf', 'Data Science 101 Statistics Overview.pdf', 'ODLU_04_2023_Odluka o dodjeli službenih mobitela 2023.pdf', 'PRAV_111_o izmjenama i dopunama Pravilnika o radu.pdf', 'PRAV_122_Pravilnik o korištenju budžeta za edukacije u 2023.pdf', 'PRAV_129_Pravilnik o službenim putovanjima_FV.pdf', 'PRAV_99_Pravilnik o radu 1017.pdf']


In [7]:
# list only docx files in "../data/ foler
doc_files = os.listdir("../data/")
doc_files = [f for f in doc_files if f.endswith(".docx")]
print(doc_files)

['Obrazac najave službenog putovanja.docx', 'Procedura_uvođenja_zaposlenika_u_rad_(za_mentorirane)_0.7.docx']


In [24]:
# list only pptx files in "../data/ foler
ppt_files = os.listdir("../data/")
ppt_files = [f for f in ppt_files if f.endswith(".pptx")]
print(ppt_files)

['Prezentacija rezultata ankete o benefitima i akcijski plan.pptx', 'Vodič za snalaženje za nove zaposlenike v02122020 (1).pptx']


## PDF data

In [9]:
# load the list of pdf files in ../data folder and loop through each file and load it
pdf_files = os.listdir("../data/")
pdf_file_paths = [os.path.join('../data/', f) for f in os.listdir('../data/') if f.endswith('.pdf')]
print(pdf_file_paths)

for pdf_file_path in pdf_file_paths:
    loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()
    print(len(documents))

    # text splitter
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1800,
            chunk_overlap=180,
            length_function = len
        )
    print(text_splitter)

    docs = text_splitter.split_documents(documents)
    print(len(docs))

    pi_db = Chroma.from_documents(
        documents=docs, 
        # embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
        # persist_directory="../pi_docs_hf/",

        embedding = OpenAIEmbeddings(),
        persist_directory="../pi_docs/",
    )
    
    pi_db.persist()
    print('\n\nCollection count is:', pi_db._collection.count())


['../data/Petar Zecevic_doktorski-pz.pdf']
114
175


Collection count is: 314


## Word documents data

In [7]:
!pip install docx2txt


  Using cached docx2txt-0.8-py3-none-any.whl


In [7]:
# load the list of pdf files in ../data folder and loop through each file and load it
from langchain.document_loaders import Docx2txtLoader
pdf_files = os.listdir("../data/")
pdf_file_paths = [os.path.join('../data/', f) for f in os.listdir('../data/') if f.endswith('.docx')]
print(pdf_file_paths)

for pdf_file_path in pdf_file_paths:
    loader = Docx2txtLoader(pdf_file_path)
    documents = loader.load()
    print(len(documents))

    # text splitter
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1800,
            chunk_overlap=180,
            length_function = len
        )
    print(text_splitter)

    docs = text_splitter.split_documents(documents)
    print(len(docs))

    pi_db = Chroma.from_documents(
        documents=docs, 
        embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
        # persist_directory="../pi_docs_hf/",

        # embedding = OpenAIEmbeddings(),
        persist_directory="../pi_docs_hf/",
    )
    
    pi_db.persist()
    print('\n\nCollection count is:', pi_db._collection.count())


['../data/Obrazac najave službenog putovanja.docx']
1
1


Collection count is: 215


## PPT data

In [8]:
# load the list of pdf files in ../data folder and loop through each file and load it
from langchain.document_loaders import UnstructuredPowerPointLoader

pdf_files = os.listdir("../data/")
pdf_file_paths = [os.path.join('../data/', f) for f in os.listdir('../data/') if f.endswith('.pptx')]
print(pdf_file_paths)

for pdf_file_path in pdf_file_paths:
    loader = UnstructuredPowerPointLoader(pdf_file_path)
    # loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()
    print(len(documents))

    # text splitter
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1800,
            chunk_overlap=180,
            length_function = len
        )
    print(text_splitter)

    docs = text_splitter.split_documents(documents)
    print(len(docs))

    pi_db = Chroma.from_documents(
        documents=docs, 
        embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
        # persist_directory="../pi_docs_hf/",

        # embedding = OpenAIEmbeddings(),
        persist_directory="../pi_docs_hf/",
    )
    
    pi_db.persist()
    print('\n\nCollection count is:', pi_db._collection.count())


['../data/Prezentacija rezultata ankete o benefitima i akcijski plan.pptx', '../data/Službena putovanja_PRAV-129-23_.pptx', '../data/Vodič za snalaženje za nove zaposlenike v02122020.pptx']
1
3


Collection count is: 218
1
3


Collection count is: 221
1
25


Collection count is: 246


In [70]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.sitemap import SitemapLoader

# Load blog post
loader = SitemapLoader(web_path="https://inteligencija.com/en/sitemap_index.xml")



In [73]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [74]:
data = loader.load()

Fetching pages: 100%|##########| 701/701 [12:24<00:00,  1.06s/it]
c:\Users\ggvozden\Anaconda3\envs\pi_assistant\lib\site-packages\bs4\builder\__init__.py:311: RuntimeWarning: coroutine 'WebBaseLoader.fetch_all' was never awaited
  universal = self.cdata_list_attributes.get('*', [])


In [75]:
print(len(data))

701


In [93]:
print(data[0].page_content)

 Bird Academy 2016 – Poslovna inteligencija                         Skip to content     ServicesImplementation servicesOLAP & ReportingBusiness / Data AnalysisData IntegrationData EngineeringPlanning & Financial ConsolidationData Governance, Data Quality & Master Data ManagementData MigrationData ScienceStrategic ICT consultingEducationsSolutionsDWH ModelsPi AnalyticsPi ProfitabilityPlanning for NonprofitsConQ Content AnalyticsSynQXBRLpointPartnersTechnology PartnersSolution partnersNewsAbout usCompany ProfileOur clientsSelected projectsSocial responsibilityPrizes and awardsEU projectsContactsBlogCareers ServicesImplementation servicesOLAP & ReportingBusiness / Data AnalysisData IntegrationData EngineeringPlanning & Financial ConsolidationData Governance, Data Quality & Master Data ManagementData MigrationData ScienceStrategic ICT consultingEducationsSolutionsDWH ModelsPi AnalyticsPi ProfitabilityPlanning for NonprofitsConQ Content AnalyticsSynQXBRLpointPartnersTechnology PartnersSolut

In [77]:

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1700, chunk_overlap=250)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
pi_db = Chroma.from_documents(
    documents=splits,
    embedding = embedding,
    persist_directory="../pidata/",
    )
    
pi_db.persist()

    # embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2") 

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-A7gZ9W3yHIM3EPIb7mA5nWvV on tokens per min. Limit: 1000000 / min. Current: 796413 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-A7gZ9W3yHIM3EPIb7mA5nWvV on tokens per min. Limit: 1000000 / min. Current: 667986 / min. Contact us through our help center at help.openai.com if you continue to have issues..


# Document Splitting
Initialize Recursive Text Splitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2300,
            chunk_overlap=300,
            length_function = len
        )
print(text_splitter)


In [10]:

docs = text_splitter.split_documents(documents)
print(len(docs))

453


Let's make the chunks more readable

In [94]:
print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


Document 1:

PRAV -99/17 Pravilnik o radu  1 / 25 
 
 
  
 
 
 
 
Broj: PRAV -99/17 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
PRAVILNIK O RADU
----------------------------------------------------------------------------------------------------
Document 2:

PRAV -99/17 Pravilnik o radu  2 / 25 
 
 
 SADRŽAJ  
 
1. OPĆE ODREDBE  ................................ ................................ ................................ ................................ ..............................  3 
2. ORGANIZACIJA RADA I SISTEMATIZACIJA POSL OVA  ................................ ................................ ................................ .. 4 
3. ZASNIVANJE RADNOG OD NOSA  ................................ ................................ ................................ ................................ .... 4 
4. OSOBNI PODACI  ................................ ................................ ................................ ................................ ..............................  5
-----

# Embeddings and Vectorstore

Create Chroma db vectorstore with HuggingFace embeddings

In [ ]:
from langchain.vectorstores import FAISS, Chroma

pi_db = Chroma.from_documents(
    documents=docs,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"),
    persist_directory="../pidata/",
)
pi_db.persist()


Create Chroma db vectorstore with OpenAI embeddings

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

pi_db = Chroma.from_documents(
    documents=docs,
    embedding = OpenAIEmbeddings(),
    persist_directory="../pidata/",
    # collection_name="pdf_data",
)
pi_db.persist()

In [12]:
print('\n\nCollection count is:', pi_db._collection.count())



Collection count is: 453


## Load financial report vectorstore from disk

In [12]:
embedding_function = OpenAIEmbeddings()
pi_db = Chroma(
    persist_directory="../pidata/", 
    embedding_function=embedding_function,
    # collection_name="pdf_data",
    )

In [13]:
print('\n\nCollection count is:', pi_db._collection.count())



Collection count is: 453


## Retrieval

Similarity search - get the k most similar results to the query/question/message

In [43]:
def pretty_chunks(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [78]:
query = "Čime se bavi Poslovna inteligencija?"

In [79]:
results_ss = pi_db.similarity_search(query, k=3)
pretty_chunks(results_ss)

Document 1:

Poslovna Inteligencija, autor na Poslovna inteligencija                         Skip to content     ServicesImplementation servicesOLAP & ReportingBusiness / Data AnalysisData IntegrationData EngineeringPlanning & Financial ConsolidationData Governance, Data Quality & Master Data ManagementData MigrationData ScienceStrategic ICT consultingEducationsSolutionsDWH ModelsPi AnalyticsPi ProfitabilityPlanning for NonprofitsConQ Content AnalyticsSynQXBRLpointPartnersTechnology PartnersSolution partnersNewsAbout usCompany ProfileOur clientsSelected projectsSocial responsibilityPrizes and awardsEU projectsContactsBlogCareers ServicesImplementation servicesOLAP & ReportingBusiness / Data AnalysisData IntegrationData EngineeringPlanning & Financial ConsolidationData Governance, Data Quality & Master Data ManagementData MigrationData ScienceStrategic ICT consultingEducationsSolutionsDWH ModelsPi AnalyticsPi ProfitabilityPlanning for NonprofitsConQ Content AnalyticsSynQXBRLpointPartner

In [46]:
results_mmr=pi_db.max_marginal_relevance_search(query,k=3, fetch_k=15)
pretty_chunks(results_mmr)

Document 1:

PRAV -99/17 Pravilnik o radu  1 / 25 
 
 
  
 
 
 
 
Broj: PRAV -99/17 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
PRAVILNIK O RADU
----------------------------------------------------------------------------------------------------
Document 2:

samostalnost u njihovom određivanju.  
 
Članak 3.  
Ako je neko pravo iz radnog odnosa različito uređeno ugovorom o radu, Pravilnikom o radu, 
sporazumom sklopljenim između radničkog vijeća i Poslodavca  ili zakonom, primjenjuje se za radnika 
najpovoljnije pravo, ako Zakonom o radu ili drugim zakonom nije dru gačije određeno.  
 
Članak 4.  
Radnik je dužan:  
 radne zadatke i poslove utvrđene ugovorom o radu uredno i pravodobno izvršavati u skladu s 
naravi i vrstom rada te internim aktima Poslodavca , 
 čuvati imovinu Poslodavca  i skrbiti o njenom održavanju,  
 biti na rad nom mjestu na vrijeme,  
 u slučaju spriječenosti dolaska na posao obavijestiti o tome bez odlaganja neposrednog 
rukovoditelja,  
 čuvati ugled i poslovnu tajnu

In [47]:
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


llm = OpenAI(temperature=0, 
             model_name="gpt-3.5-turbo-instruct",
             openai_api_key=os.environ.get("OPENAI_API_KEY"),
             verbose=True
             )
                
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=pi_db.as_retriever()
)
compressed_docs = compression_retriever.get_relevant_documents(query, k=3)
pretty_chunks(compressed_docs)

c:\Users\ggvozden\Anaconda3\envs\pi_assistant\lib\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\ggvozden\Anaconda3\envs\pi_assistant\lib\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\ggvozden\Anaconda3\envs\pi_assistant\lib\site-packages\langchain\chains\llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

PRAV -99/17 Pravilnik o radu
----------------------------------------------------------------------------------------------------
Document 2:

Pravilnik o radu
----------------------------------------------------------------------------------------------------
Document 3:

Pravilnik o radu (u daljnjem tekstu: Pravilnik), sukladno Zakonu o radu, uređuju se prava,  
obveze i odgovornosti radnika iz radnog odnosa  kao i prava i obveze Poslovne i nteligencije d.o.o. (u 
daljnjem tekstu: Poslodavac ) prema radniku te postupa k i mjere za zaštitu dostojanstva radnika i druga 
pitanja važna za radnike zaposlene kod Poslodavca  koja nisu uređena ugovorom o radu ili drugim 
aktima Poslodavca.   

Pravilnik se neposredno primjenjuje na sve radnike kod Poslodavca , članove uprave, rukovodeće osobe 
i ostale radnike koji su sklopili ugovor o radu na određeno ili neodređeno vrijeme, s punim, skraćenim ili 
nepunim radnim vremenom, koji rad obavljaju u prostoru Poslodavca , na drugom mj

Initialize LLM model 

In [22]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, 
             model_name="gpt-3.5-turbo-instruct",
             openai_api_key=os.environ.get("OPENAI_API_KEY"),
             verbose=True
             )

# Question & Answering
### Extraction through prompting

In [97]:
question = "Koji je rok za mirno rješavanje spora? "

qa_chain = RetrievalQA.from_chain_type(llm, retriever=pi_db.as_retriever(), verbose=True, return_source_documents=True)
result_metrics = qa_chain({"query": question})
print(result_metrics["result"])



> Entering new RetrievalQA chain...

> Finished chain.
The deadline for peaceful resolution of a dispute is fifteen days from the completion of that procedure.


In [98]:
# question = "Odgovori na hrvatskom tko su članovi Uprave tvrtke Poslovna inteligencija"
question = "Who are the board members of Poslovna"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=pi_db.as_retriever(), verbose=True, return_source_documents=True)
result_metrics = qa_chain({"query": question})
print(result_metrics["result"])



> Entering new RetrievalQA chain...

> Finished chain.
I don't know.


## Extract the metrics information from income statement

In [107]:
query = "Describe what is AI4VET4AI project and why is it important??"

template = '''Imaš ulogu stručnjaka za odnose s javnošću u tvrtki te marketing i promociju. 
Upoznat si s osnovnim pojmovima i znaš kako se odvija proces komuniciranja. 
U ovom slučaju odgovaraš na pitanja zaposlenika s obzirom na kontekst. Odgovore pišeš u obliku pitanja i odgovora.
Odgovore temeljiš na kontekstu i na pitanju.
Odgovori na hrvatskom jeziku.

{context}
Pitanje: {query}
Odgovor:'''


QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

from langchain.llms import OpenAI
llm = OpenAI(
    temperature=0, 
    model_name="gpt-3.5-turbo",
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    verbose=True
                )

qa_chain = RetrievalQA.from_chain_type(llm, retriever=pi_db.as_retriever(), verbose=True, return_source_documents=True)
result = qa_chain({"query": query})



> Entering new RetrievalQA chain...

> Finished chain.


In [108]:
result

{'query': 'Describe what is AI4VET4AI project and why is it important??',
 'result': 'The AI4VET4AI project is a project that aims to support the growth of an AI-skilled workforce in Europe, with a special emphasis on VET (Vocational Education and Training) teachers and learners. It addresses the need for trained workers in the field of AI. The project is important because it recognizes the increasing importance of AI in various industries and aims to ensure that there are enough skilled workers who can effectively utilize AI technology. By focusing on VET teachers and learners, the project aims to provide the necessary training and education to equip individuals with the skills needed to succeed in the field of AI.',
 'source_documents': [Document(page_content="team members. AI4VET4AI project\xa0addresses the need for trained workers in the field of AI, by supporting the growth of an AI-skilled workforce in Europe, with a special emphasis on VET teachers and learners.   Read More    0

In [109]:
print('\nAnswer:\n\n', result["result"])


Answer:

 The AI4VET4AI project is a project that aims to support the growth of an AI-skilled workforce in Europe, with a special emphasis on VET (Vocational Education and Training) teachers and learners. It addresses the need for trained workers in the field of AI. The project is important because it recognizes the increasing importance of AI in various industries and aims to ensure that there are enough skilled workers who can effectively utilize AI technology. By focusing on VET teachers and learners, the project aims to provide the necessary training and education to equip individuals with the skills needed to succeed in the field of AI.


In [110]:
# print the result in a more readable format
print('\nAnswer:\n\n', result["result"].replace('\n', ' '))


Answer:

 The AI4VET4AI project is a project that aims to support the growth of an AI-skilled workforce in Europe, with a special emphasis on VET (Vocational Education and Training) teachers and learners. It addresses the need for trained workers in the field of AI. The project is important because it recognizes the increasing importance of AI in various industries and aims to ensure that there are enough skilled workers who can effectively utilize AI technology. By focusing on VET teachers and learners, the project aims to provide the necessary training and education to equip individuals with the skills needed to succeed in the field of AI.


In [139]:
# Pretty print the result
from pprint import pprint
pprint(result)  


{'query': 'Describe what is AI4VET4AI project and why is it important??',
 'result': 'The AI4VET4AI project is a project that aims to support the growth '
           'of an AI-skilled workforce in Europe, with a special emphasis on '
           'VET (Vocational Education and Training) teachers and learners. It '
           'addresses the need for trained workers in the field of AI. The '
           'project is important because it recognizes the increasing '
           'importance of AI in various industries and aims to ensure that '
           'there are enough skilled workers who can effectively utilize AI '
           'technology. By focusing on VET teachers and learners, the project '
           'aims to provide the necessary training and education to equip '
           'individuals with the skills needed to succeed in the field of AI.',
 'source_documents': [Document(page_content="team members. AI4VET4AI project\xa0addresses the need for trained workers in the field of AI, by supp

In [138]:
# Print source documents
print('\nSource documents:\n\n', result["source_documents"][0].metadata)


Source documents:

 {'lastmod': '2021-04-18T08:36:09+00:00', 'loc': 'https://inteligencija.com/en/industrije/telecommunications/', 'source': 'https://inteligencija.com/en/industrije/telecommunications/'}
